# How to do an algorithm visualization using the package?
There are 3 major steps one needs to do:
* Implement visualization function for single "state" of the algorithm
* Implement the algorithm itself dumping the states with implemented function
* Use `animate_list` from `interactive_visualization.animation_utils` to get an interactive visualization

## Example #1: quicksort
Here we use a simple text visualization based on `IPython.display.Code`. Note that this way we prevent regular frontends from messing the whitespace structure. The state can be described by the following parameters:
* `array` - array to sort
* `left, right` - bounds of the current recursive call
* `x` - division element
* `p, q` - two pointers used for division into two subarrays with values less then and greater then `x` respectively
Visualization will mark `left, right` as square brackets and pointers `p, q` as arrows. 

In [1]:
from IPython.display import Code
def qsort_state(array, left, right, x, p, q):
    extended_array = list(map(str, array[:left])) + ['['] + list(map(str, array[left: right])) + [']'] + list(map(str, array[right:]))
    offset_x = sum(list(map(len, extended_array[:left]))) + left + 2
    zero_line = ''.join([' ' for i in range(offset_x)]) + f'x = {x}'
    first_line = ' '.join(extended_array)
    offset_p = sum(list(map(len, extended_array[:p + 1]))) + p + 1 + len(extended_array[p + 1]) // 2
    offset_q = sum(list(map(len, extended_array[:q + 1]))) + q + 1 + len(extended_array[q + 1]) // 2
    second_line = ''.join([' ' if i != offset_p and i != offset_q else '↑' for i in range(len(first_line))])

    return Code(zero_line + '\n' + first_line + '\n' + second_line)

In [2]:
qsort_state([1, 2, 3, 4, 5, 6], 2, 5, 4, 2, 4)

x = 4
1 2 [ 3 4 5 ] 6
      ↑   ↑

Now lets implement the algorithm itself. Compared to quicksort itself we add an additional list parameter to dump states and state dumping insertions.

In [3]:
import random

def qsort(array, left, right, states):
    if right - left <= 1:
        return
    x = array[random.randint(left, right - 1)]
    p = left
    q = right - 1
    states.append(qsort_state(array, left, right, x, p, q))
    while p <= q:
        while array[p] < x:
            p += 1
            states.append(qsort_state(array, left, right, x, p, q))
        while array[q] > x:
            q -= 1
            states.append(qsort_state(array, left, right, x, p, q))
        if p <= q:
            array[p], array[q] = (array[q], array[p])
            states.append(qsort_state(array, left, right, x, p, q))
            p += 1
            q -= 1
            if p <= q:
                states.append(qsort_state(array, left, right, x, p, q))
    qsort(array, left, q + 1, states)
    qsort(array, p, right, states)   

Finally we can apply our function to some array passing an empty list as a `states` argument.

In [4]:
from interactive_visualization.animation_utils import animate_list

In [5]:
a = [234, 1, 42, 3, 15, 3, 10, 9, 2]
states = []
qsort(a, 0, len(a), states)
animate_list(states, play=True, interval=400);

interactive(children=(IntSlider(value=0, description='step', max=33), Output()), _dom_classes=('widget-interac…

Note that `steps` slider is mandatory. The player and `Next`, `prev` buttons are optional, one can specify them by `play, navigation` parameters respectively (`False, True` by default).

## Example #2: Depth first search
Now lets try a more challenging example - depth first search. Here we will use a `graphviz` package for visualization. Moreover, in this example the state of the algorithm is harder to build from scratch every time, rather we'll modify it according to algorithm progress. The package provides additional `graph_utils` wrapper over `graphviz`.

In [6]:
from interactive_visualization.graph_utils import Graph, Arc, Node

Description of a DFS state consists of the following:
* Green arcs are the ones we moved in the forward direction but haven't returned yet
* Green verices are the ones we visited but haven't left yet
* Red arcs are the ones we moved in the forward direction and have returned from them
* Red vertices are the ones we visited and have already left
* Blue arcs are the ones we ignored since they led to a vertice we've already visited
At the algorithm's termination all the red arcs should form a search tree.

In [7]:
def enter_node(node):
    node.SetColor('blue')
    
def enter_arc(node, arc):
    node.SetColor('green')
    arc.attributes['style'] = 'dashed'
    arc.attributes['color'] = 'green'
    
def return_from_arc(node, arc):
    arc.attributes['style'] = 'solid'
    arc.attributes['color'] = 'red'
    node.SetColor('blue')
    
def ignore_arc(arc):
    arc.attributes['color'] = 'blue'
    
def leave_node(node):
    node.SetColor('red')

Now the search itself

In [8]:
def dfs(graph, node_id, visited, outlist, path):
    visited.add(node_id)
    path.append(node_id)
    enter_node(graph.nodes[node_id])
    outlist.append(graph.Visualize())
    for arc in graph.nodes[node_id].arcs:
        if arc.end not in visited:
            enter_arc(graph.nodes[node_id], arc)
            dfs(graph, arc.end, visited, outlist, path) 
            return_from_arc(graph.nodes[node_id], arc)
            path.append(node_id)
        else:
            ignore_arc(arc)
        outlist.append(graph.Visualize())      
    leave_node(graph.nodes[node_id])

In [9]:
arcs = [
    Arc(1, 3, 3),
    Arc(1, 4, 7),
    Arc(4, 3, 2),
    Arc(4, 5, 3),
    Arc(1, 5, 2),
    Arc(6, 4, 2),
    Arc(5, 6, 2),
    Arc(6, 7, 1),
    Arc(7, 2, 7),
    Arc(4, 2, 2),
    Arc(3, 2, 5)
]

In [10]:
# If the following code fails to execute `dot`
# It is most likely that you didn't install graphviz as a sytem package
# See https://graphviz.org/download/
graph = Graph(arcs)
visited = set()
dfs_outlist = []
path = []
dfs_outlist.append(graph.Visualize())
dfs(graph, 1, visited, dfs_outlist, path)
dfs_outlist.append(graph.Visualize())
animate_list(dfs_outlist, play=True, interval=400);

interactive(children=(IntSlider(value=0, description='step', max=19), Output()), _dom_classes=('widget-interac…